# 如何为不支持流式传输的模型禁用流式传输

<div class="admonition tip">
    <p class="admonition-title">先决条件</p>
    <p>
        本指南假设您熟悉以下内容：
        <ul>
            <li>
                <a href="https://python.langchain.com/docs/concepts/#streaming">
                    流式传输
                </a>                
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/#chat-models/">
                    聊天模型
                </a>
            </li>
        </ul>
    </p>
</div> 

一些聊天模型，包括OpenAI的新O1模型（取决于您阅读本文的时间），不支持流式传输。这在使用[astream_events API](https://python.langchain.com/docs/concepts/#astream_events)时可能会导致问题，因为它以流式模式调用模型，期望流式传输能够正常工作。

在本指南中，我们将向您展示如何为不支持流式传输的模型禁用流式传输，确保它们永远不会以流式模式调用，即使通过astream_events API调用也是如此。


In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START, END

llm = ChatOpenAI(model="o1-preview", temperature=1)

graph_builder = StateGraph(MessagesState)


def chatbot(state: MessagesState):
    return {"messages": [llm.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()


In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))


## 不禁用流式传输

现在我们已经定义了图，让我们尝试在不禁用流式传输的情况下调用`astream_events`。这应该会抛出错误，因为`o1`模型本身不支持流式传输：


In [ ]:
input = {"messages": {"role": "user", "content": "草莓中有多少个r？"}}
try:
    async for event in graph.astream_events(input, version="v2"):
        if event["event"] == "on_chat_model_end":
            print(event["data"]["output"].content, end="", flush=True)
except:
    print("不支持流式传输！")


正如我们预期的那样发生了错误，幸运的是有一个简单的修复方法！

## 禁用流式传输

现在在不修改图的情况下，让我们将模型上的[disable_streaming](https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.chat_models.BaseChatModel.html#langchain_core.language_models.chat_models.BaseChatModel.disable_streaming)参数设置为`True`，这将解决问题：


In [ ]:
llm = ChatOpenAI(model="o1-preview", temperature=1, disable_streaming=True)

graph_builder = StateGraph(MessagesState)


def chatbot(state: MessagesState):
    return {"messages": [llm.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()


现在，使用相同的输入重新运行，我们应该看不到错误：


In [ ]:
input = {"messages": {"role": "user", "content": "草莓中有多少个r？"}}
async for event in graph.astream_events(input, version="v2"):
    if event["event"] == "on_chat_model_end":
        print(event["data"]["output"].content, end="", flush=True)
